In [1]:
# plotting.py
import h5py
import numpy as np
from synthesizer.conversions import lnu_to_absolute_mag
import pandas as pd
import unyt
from unyt import erg, Hz, s
import cmasher as cmr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import sys
import glob

sys.path.append("/home/jovyan/camels/proj1/")
from setup_params import get_photometry, get_luminosity_function, get_colour_distribution, get_safe_name, get_colour_dir_name, get_magnitude_mask
from variables_config import get_config



In [2]:
import os

data_path = "/home/jovyan/camels/proj1/CV_set/CV_outputs/plots"
print(f"Checking if base path exists: {os.path.exists(data_path)}")

if os.path.exists(data_path):
    print("\nListing contents:")
    for root, dirs, files in os.walk(data_path):
        print(f"\nDirectory: {root}")
        if dirs:
            print("  Subdirectories:", dirs)
        if files:
            print("  Files:", files)

Checking if base path exists: True

Listing contents:

Directory: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots
  Subdirectories: ['IllustrisTNG', 'SIMBA', 'Astrid', 'Swift-EAGLE', 'combined']

Directory: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG
  Subdirectories: ['UVLFs', 'colours']

Directory: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs
  Subdirectories: ['intrinsic', 'attenuated']

Directory: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs/intrinsic
  Files: ['UVLF_IllustrisTNG_UV1500_z2.0_intrinsic.pdf', 'UVLF_IllustrisTNG_GALEX_FUV_z2.0_intrinsic.pdf', 'UVLF_IllustrisTNG_GALEX_NUV_z2.0_intrinsic.pdf', 'UVLF_IllustrisTNG_UV1500_z1.5_intrinsic.pdf', 'UVLF_IllustrisTNG_GALEX_FUV_z1.5_intrinsic.pdf', 'UVLF_IllustrisTNG_GALEX_NUV_z1.5_intrinsic.pdf', 'UVLF_IllustrisTNG_UV1500_z1.0_intrinsic.pdf', 'UVLF_IllustrisTNG_GALEX_FUV_z1.0_intrinsic.pdf', 'UVLF_IllustrisTNG_GALEX_NUV_z1.0_intrinsic.pdf', 'UVLF_IllustrisTNG_U

In [3]:

def get_simulation_color(simulation):
    """Get standard color for each simulation"""
    color_map = {
        "IllustrisTNG": "blue",
        "SIMBA": "green",
        "Astrid": "red",
        "Swift-EAGLE": "orange"
    }
    return color_map.get(simulation, "gray")


In [4]:
def load_simulation_data(simulations, config):
    """Load existing data files without creating plots."""
    all_uvlf_data = {}
    all_colour_data = {}
    
    for simulation in simulations:
        config = get_config(dataset="CV", simulation=simulation)
        sim_uvlf_data = {}
        sim_colour_data = {}
        
        # Load UVLF data
        for snap, redshift_info in config["redshift_values"].items():
            z_data = {}
            for category in ['intrinsic', 'attenuated']:
                for band in config["filters"][category]:
                    filter_system = get_safe_name(band, filter_system_only=True)
                    data_dir = os.path.join(config["lf_data_dir"][category][filter_system],
                                          get_safe_name(redshift_info['label']))
                    
                    # Modified to look for CV_* files
                    pattern = f"UVLF_CV_*_{get_safe_name(band)}_{redshift_info['label']}_{category}.txt"
                    file_list = glob.glob(os.path.join(data_dir, pattern))
                    
                    if file_list:
                        all_data = []
                        for filename in file_list:
                            try:
                                data = pd.read_csv(filename, sep='\t')
                                all_data.append(data)
                            except Exception as e:
                                print(f"Error loading {filename}: {e}")
                        
                        if all_data:
                            if category not in z_data:
                                z_data[category] = {}
                            z_data[category]['magnitude'] = all_data[0]['magnitude'].values
                            z_data[category]['phi'] = np.array([df['phi'].values for df in all_data])
            
            if z_data:
                sim_uvlf_data[redshift_info['redshift']] = z_data
        
        # Similar modification for colour data
        for band1, band2 in config["colour_pairs"]:
            filter_system = get_colour_dir_name(band1, band2)
            for category in ['intrinsic', 'attenuated']:
                for snap, redshift_info in config["redshift_values"].items():
                    data_dir = os.path.join(config["colour_data_dir"][category],
                                          filter_system,
                                          get_safe_name(redshift_info['label']))
                    
                    pattern = f"Colour_CV_*_{filter_system}_{redshift_info['label']}_{category}.txt"
                    file_list = glob.glob(os.path.join(data_dir, pattern))
                    
                    if file_list:
                        all_data = []
                        for filename in file_list:
                            try:
                                data = pd.read_csv(filename, sep='\t')
                                all_data.append(data)
                            except Exception as e:
                                print(f"Error loading {filename}: {e}")
                        
                        if all_data:
                            if f'{band1}-{band2}' not in sim_colour_data:
                                sim_colour_data[f'{band1}-{band2}'] = {}
                            if redshift_info['redshift'] not in sim_colour_data[f'{band1}-{band2}']:
                                sim_colour_data[f'{band1}-{band2}'][redshift_info['redshift']] = {}
                            sim_colour_data[f'{band1}-{band2}'][redshift_info['redshift']][category] = {
                                'colour': all_data[0]['colour'].values,
                                'distribution': np.array([df['distribution'].values for df in all_data])
                            }
        
        all_uvlf_data[simulation] = sim_uvlf_data
        all_colour_data[simulation] = sim_colour_data
    
    return all_uvlf_data, all_colour_data

In [5]:
def plot_cv_uvlf(input_dir, redshift_label, band, category, simulation):
    """Plot UVLF for CV simulations showing mean, scatter, and individual runs."""
    pattern = f"UVLF_CV_*_{get_safe_name(band)}_{redshift_label}_{category}.txt"
    files = glob.glob(os.path.join(input_dir, pattern))
    
    if not files:
        return None, None, None
    
    all_data = []
    for f in files:
        try:
            df = pd.read_csv(f, sep='\t')
            all_data.append(df)
        except Exception as e:
            continue
    
    if not all_data:
        return None, None, None
    
    magnitudes = all_data[0]['magnitude'].values
    phi_arrays = np.array([df['phi'].values for df in all_data])
    
    mean_phi = np.mean(phi_arrays, axis=0)
    std_phi = np.std(phi_arrays, axis=0)
    
    sim_color = get_simulation_color(simulation)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot individual realizations
    for phi in phi_arrays:
        ax.plot(magnitudes, phi, '-', color=sim_color, alpha=0.1)
    
    # Plot mean line
    ax.plot(magnitudes, mean_phi, '-', color=sim_color, linewidth=2, 
            label=f'{simulation} Mean UVLF')
    
    # Add error bars
    ax.errorbar(magnitudes, mean_phi, yerr=std_phi/np.sqrt(len(all_data)),
               fmt='none', ecolor=sim_color, capsize=5)
    
    ax.set_xlabel('M$_{UV}$ [AB mag]', fontsize=12)
    ax.set_ylabel('log$_{10}$ φ [Mpc$^{-3}$ mag$^{-1}$]', fontsize=12)
    ax.set_title(f'{simulation} {band} UVLF\n{category}, {redshift_label}', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    return fig, magnitudes, phi_arrays  # Return the full phi_arrays for later use

In [6]:
def plot_cv_colours(input_dir, redshift_label, band1, band2, category, simulation):
    """Plot colour distributions for CV simulations."""
    filter_system = get_colour_dir_name(band1, band2)
    pattern = f"Colour_CV_*_{filter_system}_{redshift_label}_{category}.txt"
    files = glob.glob(os.path.join(input_dir, pattern))
    
    if not files:
        return None, None, None
    
    all_data = []
    for f in files:
        try:
            df = pd.read_csv(f, sep='\t')
            all_data.append(df)
        except Exception as e:
            continue
    
    if not all_data:
        return None, None, None
    
    colours = all_data[0]['colour'].values
    dist_arrays = np.array([df['distribution'].values for df in all_data])
    
    mean_dist = np.mean(dist_arrays, axis=0)
    std_dist = np.std(dist_arrays, axis=0)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    sim_color = get_simulation_color(simulation)
    # Plot individual realizations
    for dist in dist_arrays:
        ax.plot(colours, dist, '-', color=sim_color, alpha=0.2)
    
    # Plot mean line
    ax.plot(colours, mean_dist, '-', color=sim_color, linewidth=2, 
            label=simulation)
    
    ax.set_xlabel(f'{band1} - {band2} [mag]', fontsize=12)
    ax.set_ylabel('Normalized Count', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(frameon=True)
    
    return fig, colours, dist_arrays  # Return the full dist_arrays for later use

In [7]:

def plot_individual_redshift_uvlfs(all_sims_data, redshifts, output_dir):
    """Create separate plots for each redshift's UVLF."""
    for redshift_info in redshifts:
        z = redshift_info['redshift']
        
        fig, ax = plt.subplots(figsize=(8, 6))
        plotted_sims = []
        
        for sim_name, sim_data in all_sims_data.items():
            color = get_simulation_color(sim_name)
            
            if z in sim_data:
                z_data = sim_data[z]
                
                if 'intrinsic' in z_data:
                    ax.plot(z_data['intrinsic']['magnitude'], 
                           z_data['intrinsic']['phi'],
                           '--', color=color, alpha=0.5)
                
                if 'attenuated' in z_data:
                    label = sim_name
                    ax.plot(z_data['attenuated']['magnitude'], 
                           z_data['attenuated']['phi'],
                           '-', color=color,
                           label=label)
        
        ax.set_xlabel('M$_{AB}$', fontsize=12)
        ax.set_ylabel('$\phi$ [Mpc$^{-3}$ dex$^{-1}$]', fontsize=12)
        ax.set_ylim(-5.0, -2.0)
        
        # Get current auto-set x limits
        current_xlim = ax.get_xlim()
        ax.set_xlim(left=current_xlim[0], right=-14)
        
        ax.grid(True, alpha=0.3)
        ax.set_title(f'z = {z}', fontsize=14)
        ax.tick_params(axis='both', which='major', labelsize=10)
        
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
                 fontsize=10, frameon=True)
        
        # Save individual plot
        individual_dir = os.path.join(output_dir, 'individual_redshifts')
        os.makedirs(individual_dir, exist_ok=True)
        plt.savefig(os.path.join(individual_dir, f'uvlf_z{redshift_info["label"]}.pdf'), 
                    bbox_inches='tight', dpi=300)
        plt.close()

In [8]:
def create_individual_plots(simulations, config):
    """Create individual UVLF and colour plots for each simulation."""
    all_uvlf_data = {}
    all_colour_data = {}
    
    for simulation in simulations:
        print(f"\nProcessing simulation: {simulation}")
        config = get_config(dataset="CV", simulation=simulation)
        
        # Initialize simulation data
        sim_uvlf_data = {}
        sim_colour_data = {}
        
        # Process UVLFs
        for snap, redshift_info in config["redshift_values"].items():
            print(f"\nProcessing redshift: {redshift_info['label']}")
            z_data = {}
            
            # UVLF plots
            for category in ['intrinsic', 'attenuated']:
                for band in config["filters"][category]:
                    print(f"Processing {category} {band}")
                    filter_system = get_safe_name(band, filter_system_only=True)
                    data_dir = os.path.join(config["lf_data_dir"][category][filter_system],
                                          get_safe_name(redshift_info['label']))
                    
                    fig, magnitudes, mean_phi = plot_cv_uvlf(
                        input_dir=data_dir,
                        redshift_label=redshift_info['label'],
                        band=band,
                        category=category,
                        simulation=simulation
                    )
                    
                    if fig is not None:
                        plot_output_dir = config["plots_dir"]["UVLFs"][category]
                        os.makedirs(plot_output_dir, exist_ok=True)
                        output_file = os.path.join(
                            plot_output_dir,
                            f"UVLF_{simulation}_{get_safe_name(band)}_{redshift_info['label']}_{category}.pdf"
                        )
                        print(f"Saving UVLF plot to: {output_file}")
                        fig.savefig(output_file, bbox_inches='tight', dpi=300)
                        plt.close(fig)
                    
                    if magnitudes is not None and mean_phi is not None:
                        if category not in z_data:
                            z_data[category] = {}
                        z_data[category]['magnitude'] = magnitudes
                        z_data[category]['phi'] = mean_phi
            
            sim_uvlf_data[redshift_info['redshift']] = z_data

            # Colour plots
            for category in ['intrinsic', 'attenuated']:
                for band1, band2 in config["colour_pairs"]:
                    print(f"Processing colours {band1}-{band2} for {category}")
                    filter_system = get_colour_dir_name(band1, band2)
                    data_dir = os.path.join(config["colour_data_dir"][category],
                                          filter_system,
                                          get_safe_name(redshift_info['label']))
                    
                    fig, colours, mean_dist = plot_cv_colours(
                        input_dir=data_dir,
                        redshift_label=redshift_info['label'],
                        band1=band1,
                        band2=band2,
                        category=category,
                        simulation=simulation
                    )
                    
                    if fig is not None:
                        plot_output_dir = config["plots_dir"]["colours"][category]
                        os.makedirs(plot_output_dir, exist_ok=True)
                        output_file = os.path.join(
                            plot_output_dir,
                            f"Colour_{simulation}_{filter_system}_{redshift_info['label']}_{category}.pdf"
                        )
                        print(f"Saving colour plot to: {output_file}")
                        fig.savefig(output_file, bbox_inches='tight', dpi=300)
                        plt.close(fig)
                    
                    if colours is not None and mean_dist is not None:
                        if f'{band1}-{band2}' not in sim_colour_data:
                            sim_colour_data[f'{band1}-{band2}'] = {}
                        if redshift_info['redshift'] not in sim_colour_data[f'{band1}-{band2}']:
                            sim_colour_data[f'{band1}-{band2}'][redshift_info['redshift']] = {}
                        sim_colour_data[f'{band1}-{band2}'][redshift_info['redshift']][category] = {
                            'colour': colours,
                            'distribution': mean_dist
                        }
        
        print(f"\nStoring data for {simulation}")
        all_uvlf_data[simulation] = sim_uvlf_data
        all_colour_data[simulation] = sim_colour_data
    
    return all_uvlf_data, all_colour_data


In [9]:
def plot_combined_uvlf(all_sims_data, redshifts, output_dir):
    """Create a multi-panel plot showing UVLFs for all simulations at different redshifts."""
    num_redshifts = len(redshifts)
    fig, axes = plt.subplots(1, num_redshifts, figsize=(20, 6))
    if num_redshifts == 1:
        axes = [axes]
    
    plotted_sims = []
    
    for ax_idx, (ax, redshift_info) in enumerate(zip(axes, redshifts)):
        z = redshift_info['redshift']
        
        for sim_name, sim_data in all_sims_data.items():
            if z not in sim_data:
                continue
                
            color = get_simulation_color(sim_name)
            z_data = sim_data[z]
            
            # Plot both intrinsic and attenuated data
            for category in ['intrinsic', 'attenuated']:
                if category not in z_data:
                    continue
                    
                linestyle = '--' if category == 'intrinsic' else '-'
                alpha_individual = 0.05
                alpha_mean = 0.5 if category == 'intrinsic' else 1.0
                
                magnitudes = z_data[category]['magnitude']
                phi_arrays = z_data[category]['phi']
                
                # Plot individual realizations
                for phi in phi_arrays:
                    ax.plot(magnitudes, phi, linestyle, color=color, alpha=alpha_individual)
                
                # Calculate and plot mean with error bands
                mean_phi = np.mean(phi_arrays, axis=0)
                std_phi = np.std(phi_arrays, axis=0)
                
                # Only add label in first panel for attenuated lines
                label = sim_name if category == 'attenuated' and sim_name not in plotted_sims else None
                ax.plot(magnitudes, mean_phi, linestyle, color=color, 
                       alpha=alpha_mean, linewidth=2, label=label)
                
                ax.fill_between(magnitudes, mean_phi - std_phi, mean_phi + std_phi,
                              color=color, alpha=0.1)
                
                if label:
                    plotted_sims.append(sim_name)
        
        # Customize axis
        ax.set_xlabel('M$_{UV}$ [AB mag]', fontsize=10)
        if ax == axes[0]:
            ax.set_ylabel('log$_{10}$ $\phi$ [Mpc$^{-3}$ mag$^{-1}$]', fontsize=10)
            # Add legend to first panel
            ax.legend(loc='upper left', fontsize=9, frameon=True)
        
        ax.set_ylim(-6.0, -2.0)
        ax.set_xlim(-26.5, -14.5)
        
        ax.grid(True, alpha=0.3)
        ax.text(0.05, 0.95, f'z = {z}', transform=ax.transAxes, 
                fontsize=10, ha='left', va='top')
        ax.tick_params(axis='both', which='major', labelsize=8)
    
    plt.subplots_adjust(wspace=0.1)
    plt.savefig(os.path.join(output_dir, 'combined_uvlf.pdf'), 
                bbox_inches='tight', dpi=300)
    plt.close()

In [10]:
def plot_combined_colours(all_sims_data, redshifts, output_dir, colour_pairs):
    """Create a multi-panel plot showing colour distributions for all simulations at different redshifts."""
    num_redshifts = len(redshifts)
    fig, axes = plt.subplots(1, num_redshifts, figsize=(20, 6))
    if num_redshifts == 1:
        axes = [axes]
    
    colour_pair = colour_pairs[0]
    colour_key = f"{colour_pair[0]}-{colour_pair[1]}"
    
    plotted_sims = []
    
    for ax_idx, (ax, redshift_info) in enumerate(zip(axes, redshifts)):
        z = redshift_info['redshift']
        
        for sim_name, sim_data in all_sims_data.items():
            if colour_key not in sim_data:
                continue
                
            color = get_simulation_color(sim_name)
            if z in sim_data[colour_key]:
                z_data = sim_data[colour_key][z]
                
                for category in ['attenuated']:
                    if category not in z_data:
                        continue
                        
                    colours = z_data[category]['colour']
                    dist_arrays = z_data[category]['distribution']
                    
                    for dist in dist_arrays:
                        ax.plot(colours, dist, '-', color=color, alpha=0.05)
                    
                    mean_dist = np.mean(dist_arrays, axis=0)
                    std_dist = np.std(dist_arrays, axis=0)
                    
                    # Only add label in first panel
                    label = sim_name if sim_name not in plotted_sims else None
                    ax.plot(colours, mean_dist, '-', color=color, linewidth=2, label=label)
                    ax.fill_between(colours, mean_dist - std_dist, mean_dist + std_dist,
                                  color=color, alpha=0.1)
                    
                    if label:
                        plotted_sims.append(sim_name)
        
        ax.set_xlabel('GALEX FUV - GALEX NUV [mag]', fontsize=10)
        if ax == axes[0]:
            ax.set_ylabel('Normalized Count', fontsize=10)
            # Add legend to first panel
            ax.legend(loc='upper right', fontsize=9, frameon=True)
        
        ax.set_xlim(-0.5, 3.5)
        ax.set_ylim(0, 2.0)
        
        ax.grid(True, alpha=0.3)
        ax.text(0.05, 0.95, f'z = {z}', transform=ax.transAxes, 
                fontsize=10, ha='left', va='top')
        ax.tick_params(axis='both', which='major', labelsize=8)
    
    plt.subplots_adjust(wspace=0.1)
    plt.savefig(os.path.join(output_dir, 'combined_colours.pdf'), 
                bbox_inches='tight', dpi=300)
    plt.close()

In [11]:
def main():
    simulations = ["IllustrisTNG", "SIMBA", "Astrid", "Swift-EAGLE"]
    config = get_config(dataset="CV", simulation=simulations[0])
    base_plots_dir = os.path.join("/home/jovyan/camels/proj1/CV_set/CV_outputs/plots")
    
    # Choose what you want to do
    MAKE_INDIVIDUAL_PLOTS = False
    MAKE_COMBINED_PLOTS = True
    
    # Load the data
    loaded_data = load_simulation_data(simulations, config)
    
    # Create plots based on flags
    if MAKE_INDIVIDUAL_PLOTS:
        # Call with just the two arguments it expects
        create_individual_plots(simulations, config)
    
    if MAKE_COMBINED_PLOTS:
        combined_plot_dir = os.path.join(base_plots_dir, "combined")
        os.makedirs(combined_plot_dir, exist_ok=True)
        
        # Create combined plots
        plot_combined_uvlf(loaded_data[0],  # UVLF data 
                         sorted(config["redshift_values"].values(), key=lambda x: x['redshift']),
                         combined_plot_dir)
        
        plot_combined_colours(loaded_data[1],  # Colour data
                           sorted(config["redshift_values"].values(), key=lambda x: x['redshift']),
                           combined_plot_dir,
                           config["colour_pairs"])

if __name__ == "__main__":
    main()